## Задание 1


Будем парсить страницу со свежеми новостям на habr.com/ru/all/.  
Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:  
KEYWORDS = ['python', 'парсинг']  
Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).  
В итоге должен формироваться датафрейм со столбцами: <дата> - <заголовок> - <ссылка>.  

In [1]:
KEYWORDS = ['python', 'парсинг']

In [2]:
import time
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd

time.sleep(0.2)

In [3]:
req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(req.text, 'html.parser')
posts = soup.find_all('li', class_='content-list__item_post')

In [4]:
now = datetime.now()
today = '{:%d-%m-%Y}'.format(now, now.day)
yesterday = '{:%d-%m-%Y}'.format(now, now.day - 1)

df = pd.DataFrame() 

for post in posts:
    post_attrs = post.attrs
    try:
        post_id = post_attrs['id']
        post_id = str(post_id.split('_')[-1])
    except:
        continue
        
    link = 'https://habr.com/ru/post/' + post_id
    titles = post.find_all('a', class_='post__title_link')
    
    for title in titles:
        title = title.text.lower()
        
    date_post = post.find('span', class_='post__time')
    
    for date in date_post:
        if 'сегодня' in date:
            date = today
        elif 'вчера' in date:
            date = yesterday
            
    preview_texts = post.find_all('div', class_='post__text')
    
    for preview_text in preview_texts:
        preview_text = preview_text.text.lower()
        
        for desired in KEYWORDS:
            if desired in preview_text:
                req2 = requests.get(link)
                soup2 = BeautifulSoup(req2.text, 'html.parser')
                full_post = soup2.find_all('div', class_='post__text')
                for full_text in full_post:
                    row = {'date': date, 'title': title, 'link': link}
                    df = pd.concat([df, pd.DataFrame([row])])
                    
df.head()

,date,title,link
0,19-07-2020,массивы bash,https://habr.com/ru/post/511608



## Задание 2.

Обязательная часть Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса   Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:   
EMAIL = [xxx@x.ru, yyy@y.com]
В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>.  
Дополнительная часть (необязательная) Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте. Документация к API VK: https://vk.com/dev/methods  

In [5]:
EMAIL = ['babaninami@icloud.com', 'sofi157@rambler.ru']

In [6]:
for email in EMAIL:
    post_req = requests.post('https://digibody.avast.com/v1/web/leaks', json = {'email' : email})
    avast = pd.DataFrame(post_req.json()['value'])
avast.head()

,leak_id,username,domain,passwords,leak_info,marked_resolved,leak_date,passwords_count
0,037a277e-d484-4c34-8473-904024e489e1,sofi157@rambler.ru,,"[{'password': None, 'encrypted': False}]","{'id': '037a277e-d484-4c34-8473-904024e489e1',...",False,1513900800000,1
1,8908dde7-dc5b-44a7-8720-d6090c933497,sofi157@rambler.ru,,"[{'password': None, 'encrypted': False}]","{'id': '8908dde7-dc5b-44a7-8720-d6090c933497',...",False,1535068800000,1
2,cf7afbbb-7695-4f97-b755-3d63df87828f,sofi157@rambler.ru,,"[{'password': None, 'encrypted': False}]","{'id': 'cf7afbbb-7695-4f97-b755-3d63df87828f',...",False,1549411200000,1
3,f9afdb14-e6f7-4c96-a7ed-457e95c5442a,sofi157@rambler.ru,,"[{'password': None, 'encrypted': False}]","{'id': 'f9afdb14-e6f7-4c96-a7ed-457e95c5442a',...",False,1593043200000,1
4,c0338e81-fce6-450c-ab32-559867d243c2,sofi157@rambler.ru,,"[{'password': None, 'encrypted': False}]","{'id': 'c0338e81-fce6-450c-ab32-559867d243c2',...",False,1507507200000,1


In [7]:
avast['leak_date'] = pd.to_datetime(avast['leak_date'], unit='ms')

In [8]:
avast.rename(columns={'leak_id': 'id'}, inplace=True)
avast.rename(columns={'username': 'E-mail'}, inplace=True)
avast.rename(columns={'leak_date': 'date'}, inplace=True)

In [9]:
avast = avast.loc[:, ['id', 'E-mail', 'date']]
avast.head()

,id,E-mail,date
0,037a277e-d484-4c34-8473-904024e489e1,sofi157@rambler.ru,2017-12-22
1,8908dde7-dc5b-44a7-8720-d6090c933497,sofi157@rambler.ru,2018-08-24
2,cf7afbbb-7695-4f97-b755-3d63df87828f,sofi157@rambler.ru,2019-02-06
3,f9afdb14-e6f7-4c96-a7ed-457e95c5442a,sofi157@rambler.ru,2020-06-25
4,c0338e81-fce6-450c-ab32-559867d243c2,sofi157@rambler.ru,2017-10-09


In [10]:
value = post_req.json()['value']

In [11]:
avast_ = pd.DataFrame() 
for items in value:
    for item in items.values():
        avast_ = pd.concat([avast_, pd.DataFrame([item])])

In [12]:
avast_.head()

,0,date,description,domains,id,media_references,number_of_entries,picture_url,service_name,source_references,title
0,037a277e-d484-4c34-8473-904024e489e1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,sofi157@rambler.ru,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,"{'password': None, 'encrypted': False}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,1.513901e+12,The proliferation of stolen or leaked database...,[],037a277e-d484-4c34-8473-904024e489e1,[],1.335492e+09,None,None,[],Combolist of 1.4 Billion Credentials


In [13]:
id_ = avast_.id.dropna().reset_index()
title = avast_.title.dropna().reset_index()
description = avast_.description.dropna().reset_index()
media_references = avast_.media_references.dropna().reset_index()
id_ = id_.loc[:, ['id']]
title = title.loc[:, ['title']]
description = description.loc[:, ['description']]
media_references = media_references.loc[:, ['media_references']]
avast__ = pd.concat([id_, title], axis=1)
avast__ = pd.concat([avast__, description], axis=1)
avast__ = pd.concat([avast__, media_references], axis=1)
avast = avast.merge(avast__, how='left', on='id')
avast = avast[['E-mail', 'date', 'title', 'description']]

In [14]:
avast.head()

,E-mail,date,title,description
0,sofi157@rambler.ru,2017-12-22,Combolist of 1.4 Billion Credentials,The proliferation of stolen or leaked database...
1,sofi157@rambler.ru,2018-08-24,Jadid List from Pemiblanc.com,This combolist was compiled from a variety of ...
2,sofi157@rambler.ru,2019-02-06,Collection #4 Combo List,"On January 7, 2019, an online user named Sanix..."
3,sofi157@rambler.ru,2020-06-25,"""Cash Cloud"" USA Combo List","At an unconfirmed date, this ""Cash Cloud"" Comb..."
4,sofi157@rambler.ru,2017-10-09,Exploit.in combolist,The proliferation of stolen or leaked database...
